<a href="https://colab.research.google.com/github/mathluva/Udemy-BERT/blob/main/Udemy_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import dependencies
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [2]:
#use ! for terminal commands
!pip install bert-for-tf2 #tensorflow2 light version
!pip install sentencepiece #required for BERT-tf2

     |████████████████████████████████| 51kB 6.3MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=7a9638ce9e00e828ae4951d3163b8080d6e44429dd875d133134965c575e511b
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=bd2b488a1175d0bd1a9c3456c9a1e7d5a3d713523e7befc0f21cc2f1dc7e8e8e
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=f532688cea324547d16ecc6fbe9ae1a8e6fd76d9e471708a8d7b9944c3217b69
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 17.1MB/s 


In [3]:
try:
    %tensorflow_version 2.x #only available in Google colab
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub #used to import the weights from BERT

from tensorflow.keras import layers
import bert #installed in previous step

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x #only available in Google colab`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


# Data preprocessing


In [4]:
#load files, data preprocessing
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
#label columns
#latin1 is common for western languages
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/trainingandtestdata.zip (Unzipped Files)/training.1600000.processed.noemoticon.csv", 
    header = None,
    names = cols,
    engine = "python",
    encoding = "latin1")

In [6]:
#axis1 column data
#without inplace=True, it would be required to write data = data.drop("...")
data.drop(["id", "date","query", "user"], axis = 1, inplace = True)

In [7]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
#cleaning
#r is regEX (regexr.com for more documentation)
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text() #tweets are not usuable as standard string, need BS to extract string
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ',tweet)#anything behind @symbol with empty space, apply to tweet
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)#? means the s can be there or not
    tweet = re.sub(r"[^a-zA-Z.!?]", ' ', tweet) #keep only standard characters
    tweet = re.sub(r" +", ' ', tweet) #replace multiple sequences of white space with only one white space
    return tweet

In [9]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [10]:
#process sentiment
data_labels = data.sentiment.values
data_labels[data_labels ==4] =1 #data is using 0 and 4, replace 4 with standard 1

# Tokenization


In [11]:
#create BERT layer to have access to metadata for the tokenizer(like vocab size).
#call BERT as a layer, hub is where all pretrained models are located
#trainable = False bc we are not fine-tuning the weights
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable = False) 
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() #way to have acces to vocab
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [12]:
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

In [13]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

# Dataset Creation

In [18]:
#create padding for each batches versus entire corpus, add token at end of each sentence
#all input to have the same length in each batch
#shuffle bc data the tweets are organized by sentiment

data_with_len = [[sent, data_labels[i] ,len(sent)] 
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len) 

#lamba, anonymous function
#data sorted by length
data_with_len.sort(key=lambda x: x[2]) 

#keep only longer sentences bc shorter sentences may not 
#convey a lot a meaning after data cleanup
sorted_all = [(sent_lab[0], sent_lab[1])
                    for sent_lab in data_with_len if sent_lab[2]>7] 

In [20]:
#need from_generator bc sentences are different lengths
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all, 
                                                output_types= (tf.int32, tf.int32))

In [21]:
#print first element
next(iter(all_dataset))

(<tf.Tensor: shape=(8,), dtype=int32, numpy=
 array([11937,  1044,  4283,  2005,  2115, 14054,  2197,  2305],
       dtype=int32)>, <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [25]:
#  padding
BATCH_SIZE =32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None,  ), ()))

In [28]:
next(iter(all_batched))

(<tf.Tensor: shape=(32, 8), dtype=int32, numpy=
 array([[11937,  1044,  4283,  2005,  2115, 14054,  2197,  2305],
        [ 2001,  2652,  6229,  1045,  2439,  2000,  1037,  2611],
        [ 3199,  6712,  2015,  2428, 11891,  2123,  1056,  2027],
        [ 4205, 12838,  2131,  1037,  2613, 10474,  3531,  2063],
        [ 2026,  2047,  9927,  2000,  5326,  2026,  5855,   999],
        [ 2735,  7245, 22104,  1045,  2031, 22939, 12171, 20192],
        [ 1037,  6904,  2497,  4465,  2009,  2003,   999,  4283],
        [ 2053, 21031,  2050,  2073,  2310,  2017,  2042,  1029],
        [ 2064,  1056,  3637,  1012,  1012,  1012,  2153,  1012],
        [ 2288,  1996,  4167, 14417,  3679,  3160,  2157,   999],
        [ 8038,  2100, 14304,   999, 26478,  8609,  2015,  2767],
        [12098,  5603,   999,   999,  3225,  4280,  2574,   999],
        [ 2821,  2053,   999,  1045,  3246,  2027,  2128,  7929],
        [ 2183,  2000,  2156,  1996,  3185,  2039,  2007,  3071],
        [13718,  2053, 11082

In [31]:
#get number of batches
NB_BATCHES = math.ceil(len(sorted_all)/BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES//10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [60]:
class DCNN(tf.keras.Model):

    def __init__(self, 
                 vocab_size,
                 emb_dim = 128, 
                 nb_filters = 50,
                 FFN_units = 512,
                 nb_classes = 2, 
                 dropout_rate = 0.1,
                 training = False,
                 name = 'dcnn'):
        
        super(DCNN, self).__init__(name = name)

        #creating layers used in the model, embedded
        self.embedding = layers.Embedding(vocab_size, emb_dim)

        self.bigram = layers.Conv1D(filters = nb_filters, 
                                               kernel_size =2, 
                                                padding = 'valid', 
                                                activation = 'relu')
        
        self.trigram = layers.Conv1D(filters = nb_filters, 
                                               kernel_size =3, 
                                                padding = 'valid', 
                                                activation = 'relu')

        self.fourgram = layers.Conv1D(filters = nb_filters, 
                                               kernel_size =4, 
                                                padding = 'valid', 
                                                activation = 'relu')
        
        self.pool = layers.GlobalMaxPooling1D()

        self.dense_1= layers.Dense(units = FFN_units,
                                    activation = 'relu')
        
        self.dropout = layers.Dropout(rate = dropout_rate)
        if nb_classes:
            self.last_dense = layers.Dense(units = 1, activation = 'sigmoid')

        else: 
            self.last_dense = layers.Dense(units = nb_classes, activation = 'softmax')

    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.bigram(x)
        x_2 = self.pool(x_2) 
        x_3 = self.bigram(x)
        x_3 = self.pool(x_3) #(batch_size, nb_filters)

        merged = tf.concat([x_1, x_2, x_3], axis =-1)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output

# Training


In [61]:
VOCAB_SIZE =    len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [62]:
Dcnn = DCNN(vocab_size = VOCAB_SIZE,
                    emb_dim =   EMB_DIM,
            nb_filters = NB_FILTERS, 
            FFN_units = FFN_UNITS,
            nb_classes = NB_CLASSES,
            dropout_rate = DROPOUT_RATE)
            

In [63]:
if NB_CLASSES ==2:
    Dcnn.compile(loss = 'binary_crossentropy',
                    optimizer = 'adam',
                 metrics = ['accuracy'])

else:
    Dcnn.compile(loss = "sparse_categorical_crossentropy",
                        optimizer = 'adam',
                  metrics = ['sparse_categorical_crossentropy'])

In [64]:
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks"

ckpt = tf.train.Checkpoint(Dcnn = Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep =1)

if ckpt_manager.latest_checkpoint:
        ckpt.restore(ckpt_manager.latest_checkpoint)
        print('Latest checkpoint has been restored')

In [65]:
#custom callback, execute during training
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs = None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [ ]:
Dcnn.fit(train_dataset,
         epochs = NB_EPOCHS,
         callbacks = [MyCustomCallback()])

Epoch 1/5
    711/Unknown - 85s 58ms/step - loss: 0.5556 - accuracy: 0.6995